<h1><center> Credit Card Fraud Analysis and Prediction 👤💰</center></h1>
    

<center><img src="https://i.ibb.co/9mPwvKZ/1936a278707545.png" width="600"></center>

Hello and greeting readers in this notebook we are going to analyze and predict fraudulent transactions from the data. Right now the notebook is on initial phase where I am updating it regularly with my trial and errors of finding best solution to perform the best predictive method to detect transactions.

Following are the tasks.
* Analyzing the data
* Treating the imbalance data
* building a model

In [1]:
"libraries"
import numpy as np
import pandas as pd

from sklearn.metrics import confusion_matrix, classification_report

In [2]:
df = pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv')
df.head()

In [3]:
df.info()

In [4]:
df.describe().T

## Missing Values

In [5]:
df.isna().sum().sum()

print(f'There {df.isna().sum().sum()} Missing Values in the dataset')

we found no missing values from the data which is a good news for further exploration and modeling

<h1><center> Exploritory Data Analysis </center></h1>

In [6]:
"libraries"
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [7]:
sns.countplot(df["Class"]);

woah, there is very large diffrence between the class 0 and 1. It is an Imbalanced dataset<br>


**what is Imbalanced data?**

Imbalanced data refers to those types of datasets where the target class has an uneven distribution of observations, i.e one class label has a very high number of observations and the other has a very low number of observations. 

## Features correlation

In [8]:
plt.figure(figsize=(36,9))
sns.heatmap(df.corr(),annot=True)



<h1><center> Modeling </center></h1>

First we have to blance the data. for that we have to undersample or oversample if we do not have balanced data we will be using the recall and precision otherwise accuracy.

In [9]:
"models"
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

"metrics"
from sklearn.metrics import accuracy_score

In [10]:
X = df.drop(columns='Class', axis=1)
y = df['Class']

X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state = 42)

In [11]:
LR = LogisticRegression(solver='liblinear').fit(X_train,y_train)

Preds = LR.predict(X_train)
LR_AC = accuracy_score(Preds,y_train)

print(f'Accuracy Score = {LR_AC}')

In [12]:
Preds2 = LR.predict(X_test)
LR_AC2 = accuracy_score(Preds2,y_test)

print(f'Accuracy Score = {LR_AC2}')

In [13]:
sns.heatmap(confusion_matrix(Preds2,y_test),annot = True,);

In [14]:
print(classification_report(y_test, Preds2))

### Undersampling Technique

Undersampling is a technique to balance uneven datasets by keeping all of the data in the minority class and decreasing the size of the majority class. It is one of several techniques data scientists can use to extract more accurate information from originally imbalanced datasets.

In [15]:
legit = df[df["Class"] == 0]
fraud = df[df["Class"] == 1]


print(f'Legit: {fraud.Class.value_counts()[1]} & Fraud : {legit.Class.value_counts()[0]}')

In [16]:
ls = legit.sample(n=492)
ls.Class.value_counts()

In [17]:
df2 = pd.concat([fraud,ls], axis = 0)

In [18]:
sns.countplot(df2["Class"]);

The class has been balanced by equal values. Lets perform modeling on this new balanced data

In [19]:
X = df2.drop(columns='Class', axis=1)
y = df2['Class']

In [20]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state = 42)

In [21]:
model = LogisticRegression(solver='liblinear').fit(X_train,y_train)

In [22]:
pred = model.predict(X_train)
actr = accuracy_score(pred,y_train)
print(f'Accuracy Score = {actr}')

In [23]:
pred = model.predict(X_test)
acts = accuracy_score(pred,y_test)
error_rate = 1 - acts
print(f'Accuracy Score = {acts}')
print(f'error_rate = {error_rate}')

There is no overfitting but is the model accurate enough to predict the fraud transaction?

In [24]:
sns.heatmap(confusion_matrix(y_test,pred),
                annot = True,
                );

<h1><center>To be continued...</center>